In [1]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
from utils import calculate_returns, compute_atr, compute_macd
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

master_ticker_data = sp500.copy()
master_ticker_data.rename(columns={"Symbol":"ticker"}, inplace = True)
master_ticker_data['ticker'] = master_ticker_data['ticker'].str.replace('.', '-')

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-12-15'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-12-18 A        37.459503   40.700001   40.799999   40.110001   40.480000   
           AAPL     25.086514   28.162500   28.162500   27.665001   27.967501   
           ABBV     44.905312   67.919998   68.010002   67.199997   67.860001   
           ABT      37.827766   45.770000   45.790001   44.439999   44.709999   
           ACGL     18.643950   19.606667   19.633333   19.383333   19.563334   
...                       ...         ...         ...         ...         ...   
2024-12-13 XYL     120.599998  120.599998  121.639999  120.400002  120.930000   
           YUM     137.000000  137.000000  139.250000  136.449997  138.410004   
           ZBH     106.739998  106.739998  106.919998  105.339996  106.440002   
           ZBRA    403.589996  403.589996  408.890015  401.549988  407.299988   
           ZTS     178.179993  178.179993  181.850006  176.630005  178.979996   

Price                   volume  
date       ticker               
2014-12-18 A         3104100.0  
           AAPL    236024800.0  
           ABBV      7184700.0  
           ABT       6703700.0  
           ACGL      1897200.0  
...                        ...  
2024-12-13 XYL       1353600.0  
           YUM       1044300.0  
           ZBH       1195300.0  
           ZBRA       249200.0  
           ZTS       1650300.0  

[1228261 rows x 6 columns]

In [2]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

df['sharpe_ratio'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.sharpe_ratio(close=x))

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-12-18 A        37.459503   40.700001   40.799999   40.110001   40.480000   
           AAPL     25.086514   28.162500   28.162500   27.665001   27.967501   
           ABBV     44.905312   67.919998   68.010002   67.199997   67.860001   
           ABT      37.827766   45.770000   45.790001   44.439999   44.709999   
           ACGL     18.643950   19.606667   19.633333   19.383333   19.563334   
...                       ...         ...         ...         ...         ...   
2024-12-13 XYL     120.599998  120.599998  121.639999  120.400002  120.930000   
           YUM     137.000000  137.000000  139.250000  136.449997  138.410004   
           ZBH     106.739998  106.739998  106.919998  105.339996  106.440002   
           ZBRA    403.589996  403.589996  408.890015  401.549988  407.299988   
           ZTS     178.179993  178.179993  181.850006  176.630005  178.979996   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2014-12-18 A         3104100.0         -0.002178        NaN       NaN   
           AAPL    236024800.0         -0.004407        NaN       NaN   
           ABBV      7184700.0         -0.065783        NaN       NaN   
           ABT       6703700.0         -0.010345        NaN       NaN   
           ACGL      1897200.0         -0.000813        NaN       NaN   
...                        ...               ...        ...       ...   
2024-12-13 XYL       1353600.0          0.000050  39.345297  4.794231   
           YUM       1044300.0          0.000166  52.773993  4.888081   
           ZBH       1195300.0          0.000108  45.108724  4.674983   
           ZBRA       249200.0          0.000132  56.606318  5.948302   
           ZTS       1650300.0          0.000416  48.957056  5.166558   

Price                bb_mid   bb_high  sharpe_ratio       atr      macd  \
date       ticker                                                         
2014-12-18 A            NaN       NaN      0.628146       NaN       NaN   
           AAPL         NaN       NaN      0.949968       NaN       NaN   
           ABBV         NaN       NaN      0.642419       NaN       NaN   
           ABT          NaN       NaN      0.584895       NaN       NaN   
           ACGL         NaN       NaN      0.740809       NaN       NaN   
...                     ...       ...           ...       ...       ...   
2024-12-13 XYL     4.837139  4.880046      0.615085  1.052145 -0.670603   
           YUM     4.924070  4.960060      0.610833  0.473350  0.618421   
           ZBH     4.708279  4.741574      0.148368 -0.521394 -0.139275   
           ZBRA    5.995411  6.042521      0.617572  0.112353  0.663372   
           ZTS     5.179572  5.192586      0.705045  0.686407 -0.416899   

Price              dollar_volume  
date       ticker                 
2014-12-18 A          116.278044  
           AAPL      5921.039336  
           ABBV       322.631192  
           ABT        253.585998  
           ACGL        35.371301  
...                          ...  
2024-12-13 XYL        163.244158  
           YUM        143.069100  
           ZBH        127.586319  
           ZBRA       100.574627  
           ZTS        294.050442  

[1228261 rows x 15 columns]

In [3]:
data = df.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-01-27 A        35.752087   38.750000   39.250000   38.580002   38.700001   
           AAPL     24.304859   27.285000   28.120001   27.257500   28.105000   
           ABBV     42.031765   63.099998   63.500000   62.200001   62.330002   
           ABT      36.291531   43.680000   44.020000   43.430000   43.880001   
           ACGL     18.973595   19.953333   20.160000   19.683332   19.760000   
...                       ...         ...         ...         ...         ...   
2024-12-13 XYL     120.599998  120.599998  121.639999  120.400002  120.930000   
           YUM     137.000000  137.000000  139.250000  136.449997  138.410004   
           ZBH     106.739998  106.739998  106.919998  105.339996  106.440002   
           ZBRA    403.589996  403.589996  408.890015  401.549988  407.299988   
           ZTS     178.179993  178.179993  181.850006  176.630005  178.979996   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-01-27 A         1703500.0         -0.002277  42.551069  3.572676   
           AAPL    382274800.0         -0.007667  45.914747  3.195314   
           ABBV      6532300.0         -0.059758  41.454637  3.738412   
           ABT       4847400.0         -0.013836  39.157376  3.615258   
           ACGL      1379700.0         -0.000351  58.501939  2.967072   
...                        ...               ...        ...       ...   
2024-12-13 XYL       1353600.0          0.000050  39.345297  4.794231   
           YUM       1044300.0          0.000166  52.773993  4.888081   
           ZBH       1195300.0          0.000108  45.108724  4.674983   
           ZBRA       249200.0          0.000132  56.606318  5.948302   
           ZTS       1650300.0          0.000416  48.957056  5.166558   

Price                bb_mid  ...  sharpe_ratio       atr      macd  \
date       ticker            ...                                     
2015-01-27 A       3.624346  ...      0.628146 -1.001110 -0.615512   
           AAPL    3.237558  ...      0.949968 -0.921370 -0.295065   
           ABBV    3.780835  ...      0.642419 -0.570094 -0.687058   
           ABT     3.638065  ...      0.584895 -1.015763 -0.567777   
           ACGL    2.982360  ...      0.740809 -1.057993 -0.188747   
...                     ...  ...           ...       ...       ...   
2024-12-13 XYL     4.837139  ...      0.615085  1.052145 -0.670603   
           YUM     4.924070  ...      0.610833  0.473350  0.618421   
           ZBH     4.708279  ...      0.148368 -0.521394 -0.139275   
           ZBRA    5.995411  ...      0.617572  0.112353  0.663372   
           ZTS     5.179572  ...      0.705045  0.686407 -0.416899   

Price              dollar_volume  return_1d  return_2d  return_3d  return_6d  \
date       ticker                                                              
2015-01-27 A           60.903680  -0.010217  -0.000773  -0.007624   0.002167   
           AAPL      9291.135175  -0.035013  -0.017141  -0.009763   0.004893   
           ABBV       274.564098   0.004297   0.005352   0.000635  -0.003754   
           ABT        175.919565  -0.010646  -0.002054  -0.006106  -0.003058   
           ACGL        26.177869   0.002176   0.003780   0.001060   0.001709   
...                          ...        ...        ...        ...        ...   
2024-12-13 XYL        163.244158  -0.003882  -0.011330  -0.021106  -0.007576   
           YUM        143.069100  -0.011829  -0.005573  -0.002951  -0.001368   
           ZBH        127.586319  -0.000936  -0.000795  -0.005131  -0.003729   
           ZBRA       100.574627  -0.009935  -0.000718   0.001499  -0.000573   
           ZTS        294.050442  -0.003690   0.002846   0.002765   0.003226   

Price              return_9d  return_12d  
d

#### Download Fama-French Factors and Calculate Rolling Factor Betas.

We will introduce the Fama—French data to estimate the exposure of assets to common risk factors using linear regression.

The five Fama—French factors, namely market risk, size, value, operating profitability, and investment have been shown empirically to explain asset returns and are commonly used to assess the risk/return profile of portfolios. Hence, it is natural to include past factor exposures as financial features in models.

We can access the historical factor returns using the pandas-datareader and estimate historical exposures using the RollingOLS rolling linear regression.

In [4]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1d']).sort_index()

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1d
date       ticker                                                   
2015-03-31 A      -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.004075
           AAPL   -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.015352
           ABBV   -0.0112  0.0307 -0.0038  0.0009 -0.0052   0.005151
           ABT    -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.015931
           ACGL   -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.017230
...                   ...     ...     ...     ...     ...        ...
2024-10-31 XYL    -0.0097 -0.0087  0.0089 -0.0140  0.0103  -0.057777
           YUM    -0.0097 -0.0087  0.0089 -0.0140  0.0103  -0.016718
           ZBH    -0.0097 -0.0087  0.0089 -0.0140  0.0103  -0.026318
           ZBRA   -0.0097 -0.0087  0.0089 -0.0140  0.0103  -0.005001
           ZTS    -0.0097 -0.0087  0.0089 -0.0140  0.0103  -0.019352

[40531 rows x 6 columns]

In [5]:
observations = factor_data.groupby(level=1).size()

valid_stocks = observations[observations >= 10]

factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1d
date       ticker                                                   
2015-03-31 A      -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.004075
           AAPL   -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.015352
           ABBV   -0.0112  0.0307 -0.0038  0.0009 -0.0052   0.005151
           ABT    -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.015931
           ACGL   -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.017230
...                   ...     ...     ...     ...     ...        ...
2024-10-31 XYL    -0.0097 -0.0087  0.0089 -0.0140  0.0103  -0.057777
           YUM    -0.0097 -0.0087  0.0089 -0.0140  0.0103  -0.016718
           ZBH    -0.0097 -0.0087  0.0089 -0.0140  0.0103  -0.026318
           ZBRA   -0.0097 -0.0087  0.0089 -0.0140  0.0103  -0.005001
           ZTS    -0.0097 -0.0087  0.0089 -0.0140  0.0103  -0.019352

[40512 rows x 6 columns]

Calculate Rolling Factor Betas.

In [6]:
betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1d'], 
                                     exog=sm.add_constant(x.drop('return_1d', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2015-03-31 A            NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACGL         NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2024-10-31 XYL     0.145617 -0.002586  0.044574  0.245492 -0.410235
           YUM     0.086260  0.056658  0.022026  0.210514 -0.131489
           ZBH     0.169767 -0.132473  0.044221  0.011692 -0.142913
           ZBRA    0.076819  0.158266 -0.055861  0.140365 -0.089666
           ZTS     0.087845  0.047270 -0.143117  0.203909  0.043824

[40512 rows x 5 columns]

Join the rolling factors data to the main features dataframe

In [7]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

data = (data.join(betas.groupby('ticker').shift()))

data.loc[:, factors] = data.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

data = data.drop('adj close', axis=1)

data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1213413 entries, (Timestamp('2015-01-27 00:00:00'), 'A') to (Timestamp('2024-12-13 00:00:00'), 'ZTS')
Data columns (total 25 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   close             1213413 non-null  float64
 1   high              1213413 non-null  float64
 2   low               1213413 non-null  float64
 3   open              1213413 non-null  float64
 4   volume            1213413 non-null  float64
 5   garman_klass_vol  1213413 non-null  float64
 6   rsi               1213413 non-null  float64
 7   bb_low            1213413 non-null  float64
 8   bb_mid            1213413 non-null  float64
 9   bb_high           1213413 non-null  float64
 10  sharpe_ratio      1213413 non-null  float64
 11  atr               1213413 non-null  float64
 12  macd              1213413 non-null  float64
 13  dollar_volume     1213413 non-null  float64
 14  return_1d         1213413 

In [8]:
data.head()

close       high        low       open       volume  \
date       ticker                                                            
2015-01-27 A       38.750000  39.250000  38.580002  38.700001    1703500.0   
           AAPL    27.285000  28.120001  27.257500  28.105000  382274800.0   
           ABBV    63.099998  63.500000  62.200001  62.330002    6532300.0   
           ABT     43.680000  44.020000  43.430000  43.880001    4847400.0   
           ACGL    19.953333  20.160000  19.683332  19.760000    1379700.0   

                   garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                              
2015-01-27 A              -0.002277  42.551069  3.572676  3.624346  3.676016   
           AAPL           -0.007667  45.914747  3.195314  3.237558  3.279801   
           ABBV           -0.059758  41.454637  3.738412  3.780835  3.823258   
           ABT            -0.013836  39.157376  3.615258  3.638065  3.660871   
           ACGL           -0.000351  58.501939  2.967072  2.982360  2.997649   

                   ...  return_2d  return_3d  return_6d  return_9d  \
date       ticker  ...                                               
2015-01-27 A       ...  -0.000773  -0.007624   0.002167  -0.002268   
           AAPL    ...  -0.017141  -0.009763   0.004893  -0.001093   
           ABBV    ...   0.005352   0.000635  -0.003754  -0.000509   
           ABT     ...  -0.002054  -0.006106  -0.003058  -0.002338   
           ACGL    ...   0.003780   0.001060   0.001709   0.001308   

                   return_12d    Mkt-RF       SMB       HML       RMW  \
date       ticker                                                       
2015-01-27 A        -0.004470 -0.013231 -0.040099  0.049200  0.050537   
           AAPL     -0.002072  0.073536 -0.020394 -0.015091  0.155134   
           ABBV     -0.005141  0.072633  0.045399 -0.073538  0.175086   
           ABT      -0.003286 -0.030630  0.048404  0.014877  0.151122   
           ACGL      0.000321 -0.064490 -0.056542  0.171240 -0.082922   

                        CMA  
date       ticker            
2015-01-27 A      -0.007434  
           AAPL    0.208852  
           ABBV   -0.034092  
           ABT    -0.068621  
           ACGL   -0.165375  

[5 rows x 25 columns]

### Save price data

In [9]:
data.reset_index(inplace = True)

In [10]:
data.to_parquet("data/price_data.gzip", compression='gzip', index = False)
data.to_csv("data/price_data.csv", index = False)

In [11]:
data.head()

,date,ticker,close,high,low,open,volume,garman_klass_vol,rsi,bb_low,...,return_2d,return_3d,return_6d,return_9d,return_12d,Mkt-RF,SMB,HML,RMW,CMA
0,2015-01-27,A,38.750000,39.250000,38.580002,38.700001,1703500.0,-0.002277,42.551069,3.572676,...,-0.000773,-0.007624,0.002167,-0.002268,-0.004470,-0.013231,-0.040099,0.049200,0.050537,-0.007434
1,2015-01-27,AAPL,27.285000,28.120001,27.257500,28.105000,382274800.0,-0.007667,45.914747,3.195314,...,-0.017141,-0.009763,0.004893,-0.001093,-0.002072,0.073536,-0.020394,-0.015091,0.155134,0.208852
2,2015-01-27,ABBV,63.099998,63.500000,62.200001,62.330002,6532300.0,-0.059758,41.454637,3.738412,...,0.005352,0.000635,-0.003754,-0.000509,-0.005141,0.072633,0.045399,-0.073538,0.175086,-0.034092
3,2015-01-27,ABT,43.680000,44.020000,43.430000,43.880001,4847400.0,-0.013836,39.157376,3.615258,...,-0.002054,-0.006106,-0.003058,-0.002338,-0.003286,-0.030630,0.048404,0.014877,0.151122,-0.068621
4,2015-01-27,ACGL,19.953333,20.160000,19.683332,19.760000,1379700.0,-0.000351,58.501939,2.967072,...,0.003780,0.001060,0.001709,0.001308,0.000321,-0.064490,-0.056542,0.171240,-0.082922,-0.165375


### Save data to sql db

In [12]:
import sqlite3
conn = sqlite3.connect('data/database.db')
c = conn.cursor()

In [13]:
data.to_sql('price_data', conn, if_exists='replace', index = False)
master_ticker_data.to_sql('master_ticker', conn, if_exists='replace')

503